In [65]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
from astropy.io import ascii
from utils import col_names, relevant_indices, normalise_sdss_class, ellipticity, filling_factor
import matplotlib.pyplot as plt
%matplotlib inline


In [70]:
datasets = "../datasets/SuperCOSMOS/"
colnames_relevant=[col_names[i] for i in relevant_indices]
uki823_df = ascii.read(datasets + "UKI823/sssedrpair.dat", guess=False, Reader=ascii.FastNoHeader).to_pandas()
data=uki823_df.iloc[:, relevant_indices]
data = pd.DataFrame(data.values, columns = colnames_relevant)   
data['Ellipticity']=  ellipticity(uki823_df.iloc[:,16],uki823_df.iloc[:,17])
data['Filling Factor']= filling_factor(data['AREA'], uki823_df.iloc[:,13],uki823_df.iloc[:,14])
data=normalise_sdss_class(data)

In [71]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15645 entries, 0 to 15644
Data columns (total 24 columns):
AREA              15645 non-null float64
IPEAK             15645 non-null float64
COSMAG            15645 non-null float64
ISKY              15645 non-null float64
A_U               15645 non-null float64
B_U               15645 non-null float64
THETA_U           15645 non-null float64
A_I               15645 non-null float64
B_I               15645 non-null float64
THETA_I           15645 non-null float64
BLEND             15645 non-null float64
QUALITY           15645 non-null float64
N(0,1)            15645 non-null float64
PRFMAG            15645 non-null float64
C_COSMAG          15645 non-null float64
C_PRFMAG          15645 non-null float64
RA_SDSS           15645 non-null float64
DEC_SDSS          15645 non-null float64
GMAG_SDSS         15645 non-null float64
RMAG_SDSS         15645 non-null float64
IMAG_SDSS         15645 non-null float64
CLASS_SDSS        15645 non-nu

In [63]:
print("There are {} entries and {} columns in the uki823 DataFramewhen we select the relevant columns"\
      .format(data.shape[0], data.shape[1]))


There are 15645 entries and 24 columns in the uki823 DataFramewhen we select the relevant columns


In [73]:
data.describe()

,AREA,IPEAK,COSMAG,ISKY,A_U,B_U,THETA_U,A_I,B_I,THETA_I,...,C_COSMAG,C_PRFMAG,RA_SDSS,DEC_SDSS,GMAG_SDSS,RMAG_SDSS,IMAG_SDSS,CLASS_SDSS,Ellipticity,Filling Factor
count,15645.000000,1.564500e+04,15645.000000,1.564500e+04,15645.000000,15645.000000,15645.000000,15645.000000,15645.000000,15645.000000,...,15645.000000,15645.000000,15645.000000,15645.000000,15645.000000,15645.000000,15645.000000,15645.000000,15645.000000,1.564500e+04
mean,92.001151,1.319364e+07,-20152.272867,1.808098e+07,5423.954618,4149.977245,84.416874,4291.177181,3267.532886,81.601214,...,17.481817,17.367207,174.007529,0.048635,19.528314,18.495397,17.893314,1.628444,-0.033059,inf
std,314.990581,1.817166e+07,1971.753396,4.441520e+05,3550.485593,3341.520804,57.212674,1935.854217,1881.680318,55.967697,...,1.285655,1.894035,179.404607,0.724706,2.194382,1.931732,1.773068,0.483236,0.030039,NaN
min,8.000000,1.297920e+06,-28893.000000,1.709983e+07,1633.000000,538.000000,0.000000,1551.000000,524.000000,0.000000,...,11.154000,4.781000,0.000165,-1.243447,8.985260,6.618781,6.940934,1.000000,-0.285714,1.471725e-06
25%,17.000000,2.348600e+06,-21393.000000,1.769842e+07,3434.000000,2121.000000,35.000000,3235.000000,2035.000000,30.000000,...,16.776000,16.536000,0.502112,-0.569497,18.465448,17.573668,17.055212,1.000000,-0.048045,2.697026e-05
50%,37.000000,3.912512e+06,-19703.000000,1.802993e+07,4537.000000,3286.000000,79.000000,4006.000000,3000.000000,73.000000,...,17.858000,18.019000,0.969110,0.075373,19.967171,18.960104,18.355196,2.000000,-0.026707,5.010798e-05
75%,92.000000,1.417421e+07,-18561.000000,1.845908e+07,6325.000000,5203.000000,135.000000,4777.000000,4040.000000,134.000000,...,18.471000,18.748000,359.492904,0.682301,21.087143,19.793398,19.087181,2.000000,-0.008006,1.382349e-04
max,24006.000000,1.210549e+08,-17410.000000,1.899585e+07,101032.000000,90696.000000,999.000000,59866.000000,55313.000000,179.000000,...,19.292000,19.399000,359.999933,1.273383,22.999374,24.802490,27.166468,2.000000,0.001029,inf
